In [223]:
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

Data Preprocessing

In [224]:
#text preprocesing 
#step 1 : read the file
with open('gutenberg_corpus.txt', 'r' , encoding='utf-8') as file:
    text  = file.read()

In [225]:
#tokenize and normalize
tokens = word_tokenize(text)
#remove punctuations
filtered_tokens = [word for word in tokens if any(char.isalnum() for char in word)]

In [226]:
def preprocess_word(word):
    # Remove punctuation within words
    word = ''.join(char for char in word if char not in string.punctuation)
    # Normalize by converting to lowercase
    word = word.lower()
    return word

In [227]:
#remove stop words
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in filtered_tokens if word not in stop_words]

words  = [preprocess_word(word) for word in filtered_words]
words = [word.lower() for word in words]

In [228]:
def create_word_occurences_map(words):
    word_occurences = {}
    for word in words:
        if word in word_occurences:
            word_occurences[word] += 1
        else :
            word_occurences[word] = 1
    return word_occurences

In [229]:
def create_probability_map(word_occurrences_map):
    total_occurrences = sum(word_occurrences_map.values())
    probability_map = {word: occurrences / total_occurrences for word, occurrences in word_occurrences_map.items()}
    return probability_map

In [230]:
#things which we will use , final
word_occurrences = create_word_occurences_map(words)
word_probability = create_probability_map(word_occurrences)

BUILDING THE TRIE FOR AUTOCOMPLETE

In [231]:
class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.is_end_of_word = False

In [232]:
class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
    
    def search(self, prefix):
        if not self.root.children:
            return []
        
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]
        return self._get_words_with_prefix(node, prefix)
    
    def _get_words_with_prefix(self, node, prefix):
        words = []
        if node.is_end_of_word:
            words.append(prefix)
        for char, child_node in node.children.items():
            words.extend(self._get_words_with_prefix(child_node, prefix + char))
        return words

In [233]:
words = list(word_occurrences.keys()) # words to insert in the trie

Insert the words in the trie

In [234]:
trie = Trie()
for word in words:
    trie.insert(word)

In [251]:
#find the suggestions
suggestions = trie.search('heav')
#build the suggestions probability map
suggestions_probability = {}
for sugg in suggestions:
    suggestions_probability[sugg] = word_probability[sugg]

sorted_suggested_words = sorted(suggestions_probability.items(), key=lambda x: x[1], reverse=True)
sorted_suggested_words

[('heaven', 0.000988008439132923),
 ('heavy', 0.00017948396666339305),
 ('heavens', 0.00016678462939947374),
 ('heavenly', 8.127575848908364e-05),
 ('heave', 4.23311242130644e-05),
 ('heavily', 3.1325031917667655e-05),
 ('heaved', 1.7779072169487048e-05),
 ('heaving', 1.7779072169487048e-05),
 ('heavier', 1.5239204716703185e-05),
 ('heaviness', 1.269933726391932e-05),
 ('heaviest', 5.926357389829016e-06),
 ('heavengate', 4.23311242130644e-06),
 ('heaves', 2.539867452783864e-06),
 ('heavengates', 1.6932449685225761e-06),
 ('heavers', 1.6932449685225761e-06),
 ('heaveto', 1.6932449685225761e-06),
 ('heav', 8.466224842612881e-07),
 ('heavenlyborn', 8.466224842612881e-07),
 ('heaveninsulting', 8.466224842612881e-07),
 ('heavenabiding', 8.466224842612881e-07),
 ('heavenambitious', 8.466224842612881e-07),
 ('heavengazer', 8.466224842612881e-07),
 ('heavenwarring', 8.466224842612881e-07),
 ('heavenward', 8.466224842612881e-07),
 ('heavenbanished', 8.466224842612881e-07),
 ('heavenfallen', 8.4